In [1]:
from apollo.src.prof_model.fund_netvalue import FundNetValue
from apollo.src.db_model.tbl_info import InfoTable

In [8]:
code = '005827'
start = '2020-08-04'
end = '2021-08-03'
amount = 100

fund_val = FundNetValue(code)
price_df = fund_val.read_sql()
print(InfoTable.get_by_code(code).name)

易方达蓝筹精选混合


In [9]:
start_index = price_df.loc[price_df['date'] == start].index[0]
end_index = price_df.loc[price_df['date'] == end].index[0]
buy_df = price_df.iloc[start_index : end_index] # 待买df

In [10]:
price_df.loc[price_df['date'] == end]['totvalue']

679    2.5785
Name: totvalue, dtype: float64

In [15]:
invest_week(code, start, end, amount)

易方达蓝筹精选混合
每周 1 定投，累计投入 4700 单位金额， 最终卖出 4483.31 单位金额， 收益率 -4.61% ;
每周 2 定投，累计投入 4900 单位金额， 最终卖出 4662.76 单位金额， 收益率 -4.84% ;
每周 3 定投，累计投入 4900 单位金额， 最终卖出 4663.98 单位金额， 收益率 -4.82% ;
每周 4 定投，累计投入 4900 单位金额， 最终卖出 4628.33 单位金额， 收益率 -5.54% ;
每周 5 定投，累计投入 4900 单位金额， 最终卖出 4655.94 单位金额， 收益率 -4.98% ;


In [14]:
def invest_week(code, start, end, amount):
    """
    每周定投

    :param code:    基金代码        str      '005827'
    :param start:   定投开始日       str     '2020-08-04'
    :param end:     定投结束日       str     '2021-08-03'
    :param amount:  每次投资的金额    int     100
    :return: 
    """
    fund_val = FundNetValue(code)
    price_df = fund_val.read_sql()
    print(InfoTable.get_by_code(code).name)
    
    start_index = price_df.loc[price_df['date'] == start].index[0]
    end_index = price_df.loc[price_df['date'] == end].index[0]
    buy_df = price_df.iloc[start_index : end_index] # 待买df
    sell_price = float(price_df.loc[price_df['date'] == end]['totvalue']) # 要卖那天的累计净值
    
    total = [0] * 5  # 存放对应星期合计买的份数
    count = [0] * 5  # 存放对应星期的成本

    for index, row in buy_df.iterrows():
        weekday = int(row['date'].weekday())
        # amount金额除以当日累计净值得到购买基金份额，并计算累计份额
        total[weekday] = total[weekday] + amount/float(row['totvalue']) 
        count[weekday] += amount

    for index, unit in enumerate(total):
        print(f"每周 {index+1} 定投，累计投入 {count[index]} 单位金额，",
              f"最终卖出 {round(sell_price*unit,2)} 单位金额，",
              f"收益率 {round(100*(sell_price*unit-count[index])/count[index],2)}% ;")